## 8.3.1 컨볼루션 신경망을 사용한 텍스처 합성

텍스터 합성은 한 장의 이미지를 원본으로 삼아 해당 텍스처를 재생성하는 작업이다. 이때 합성되는 이미지는 원본과 비슷해야 하지만 똑같아서는 안되며 어색한 부분과 반복적인 부분이 없어야 하며 원하는 크# 8.3.1 컨볼루션 신경망을 사용한 텍스처 합성

텍스터 합성은 한 장의 이미지를 원본으로 삼아 해당 텍스처를 재생성하는 작업이다. 이때 합성되는 이미지는 원본과 비슷해야 하지만 똑같아서는 안되며 어색한 부분과 반복적인 부분이 없어야 하며 원하는 크기로 생성 가능해야 한다.

기존의 텍스처 합성 방법 중 가장 효과적이었던 방법은 두가지였다.

첫번째로 픽셀이나 이미지를 잘게 쪼갠 Patch를 재벼열하는 방법이었다.

두번째 방법은 파라미터에 의한 텍스처 모델링이다. 먼저 원본 텍스처의 공갅거인 통곗값을 사람이 정교하게 만든 여러개의 필터로 구한다. 필터를 통과한 결과물이 같다면 같은 텍스처라고 가정한 후 이 결과물이 같아질 때까지 타깃 텍스처를 변형시키게 된다. 

여기서 필터를 컨볼루션 신경망이 자동으로 만들게 하여 네트워크가 이미지의 특징을 자동으로 추출할 수 있게 하는 것이다. 

In [ ]:

# 8.42 원본 텍스쳐 이미지 불러오기
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

style_path = tf.keras.utils.get_file('style.jpg', 'http://bit.ly/2mGfZIq')

style_image = plt.imread(style_path)
style_image = cv2.resize(style_image, dsize=(224, 224))
style_image = style_image / 255.0
plt.imshow(style_image)

tf.kears.utils.get_file() 함수로 파일을 다우로드 한 후 OpneCV로 이미지의 크길르 224,224로 조정한다. 더 큰 크기로 해도 상관 없지만, 일단 작은 크기에서 어떻게 동작하는지 확인해보기로 한다.

그 다음 타깃 텍스처로 사용할 이미지를 만든다. 타깃 텍스처는 랜덤 노이즈 이미지에서 시작한다. 

In [ ]:
# 8.43 타겟 텍스쳐 만들기
target_image = tf.random.uniform(style_image.shape)
print(target_image[0,0,:])
plt.imshow(target_image)

In [ ]:

# 8.44 VGG-19 네트워크 불러오기
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input

vgg = VGG19(include_top=False, weights='imagenet')

for layer in vgg.layers:
    print(layer.name)

전체 네트워크를 불러올 필요가 없기 때문에 include_top 인수를 False로 지정하여 마지막 Dense 레이어를 제외한 나머지 레이어를 불러와 vgg 변수에 저장했다. 

vgg 변수에 로드된 네트워크는 특징 추출기의 역할을 하는 컨볼루션 레이어와 풀링 레이어를 포함하고 있다. 이 중 어느 레이어를 이요할 것인지는 반복적인 실험을 통해 좋은 값을 얻어야 한다. 모든 레이어를 사용할 수도 있지만 일부 레이어만 사용할 수도 있다. 다만 Gram matrix가 지역적인 구저와 전체적인 구조를 모두 잡아낼 수 있도록 앞쪽과 뒤쪽의 레이어를 모두 사용하는 것이 좋다.


In [ ]:
# 8.45 특징 추출 모델 만들기
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

vgg.trainable = False
outputs = [vgg.get_layer(name).output for name in style_layers]
model = tf.keras.Model([vgg.input], outputs)

style_layers에는 네트워크 전체로 볼 떄 다섯개의 컨볼루션 레이어를 고르게 선택해서 레이어의 이름을 저장한다. 그리고 vgg-19 네트워크의 기존 가중치는 학습되지 않도록 얼려놓고, 선택된 다섯개의 레이어를 출력하는 새로운 모델을 만들었다. 

In [ ]:
# 8.46 Gram matrix 계산 함수 정의
def gram_matrix(input_tensor):
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)

먼저 입력되 ㄴ특징 추출값의 형태를 벡터로 변환시킨다. 예를 들어 첫번째 레이어인 block1_conv1을 통과한 특징 추출값의 차원수가 224,224,64이라면 이것을 맨뒤 차원인 64만 남기고 나머지를 1차원의 벡터로 만들면 차원수는 (50176, 64)가 된다.

이렇게 만든 행렬을 자기 자신의 전치 행렬과 행렬곡 하는 부분이 gram.tf.matmul(a,a,transpose_a=True)이다. matmul()은 행렬곱 함수이다. 이때 전치 행렬을 만든 다음에 행렬곱을 해도 되고 아니면 trnaspose_a라는 인수에 True값을 넣어서 행렬곱을 할 떄 전치 행렬을 자동으로 만들어서 계산해도 된다.

transpose_a가 True이기 때문에 행렬곱 계산 결과의 차원은 (64,64)가 된다.
[64,50176] * [50176, 64] = [64,64]

마지막 return 문에서 1차원 벡터의 길이인 50176으로 Gram matrix 값을 나눈다. 이렇게 벡터의 길이로 나누지 않으면 앞쪽에 오는 레이어일수록 특징 추출값의 이미지가 크기 때문에 Gram matrix 값도 커져서 영향을 준다.


In [ ]:
# 8.47 원본 텍스쳐에서 gram matrix 계산
style_image = plt.imread(style_path)
style_image = cv2.resize(style_image, dsize=(224, 224))
style_image = style_image / 255.0

style_batch = style_image.astype('float32')
style_batch = tf.expand_dims(style_batch, axis=0)
style_output = model(preprocess_input(style_batch * 255.0))

첫 세줄은 이미지의 크기를 바꾸고 픽셀 정규화를 하는 내용이다. 그 다음으로 나오는 style_batch는 이미지의 가장 앞쪽에 배치 차원을 넣어준다. 즉 (224,224,3)이었단 차원이 (1,224,224,3)으로 바뀐다. 배치 차원이 있어야 네트워크에 입력으로 넣을 수 있기 때문에 이렇게 배치 차원을  추가한다.

그 다음 model에 원본 이미지를 넣는다. preprocess_input()함수는 vgg-19 에서 이미지를 전처리하는 함수로, 0 ~ 255의 픽셀 값이 들어오는 것ㅇ르 가정하고 만들어진 함수이기 때문에 255.0을 곱한다.

style_output은 다섯 레이어를 통과한 특징 추출값으로 구성돼있다. 그 중 하나를 matplotlib.pyplot으로 출력해보자.

In [ ]:
# 8.48 원본 텍스쳐의 첫번째 특징 추출값 확인
print(style_output[0].shape)
plt.imshow(tf.squeeze(style_output[0][:,:,:,0], 0), cmap='gray')

In [ ]:
# 8.49 원본 텍스쳐의 gram matrix 계산값 만들기, 분포 확인
style_outputs = [gram_matrix(out) for out in style_output]

plt.figure(figsize=(12,10))
for c in range(5):
    plt.subplot(3,2,c+1)
    array = sorted(style_outputs[c].numpy()[0].tolist())
    array = array[::-1]
    plt.bar(range(style_outputs[c].shape[0]), array)
    plt.title(style_layers[c])
plt.show()

Gram matrix 값은 레이어마다 다르게 나오고 최댓값도 크게 차이가 난다. 각 레이어에서 ㅖ산되는 Gram matrix 값에 가중치를 곱해주는 방법으로 특정한 레이어가 큰 영향을 끼치지 못하도록 제어할 수 있지만 여기선 가중치 없이 계산한다.

타깃 텍스처를 업데이트하기 위해서는 몇가지 함수를 설정해야한다.

먼저 타겟 텍스처에서 Gram matrix를 구하는 함수가 필요하다. 위에서 Gram matrix 값을 구한 코드를 그대로 사용하며 함수로 묶어주고 인수 자리에 텍스처 이미지를 넣으면 된다. 그리고 원본 텍스처의 Gram matrix 값과 타깃 텍스처와 Grma matrix 사이의 MSE를 구하는 함수가 필요하다. 또 나오는 값이 0.0에서 1.0 사이의 컬러 값이어야 하기 때문에 그 이하나 이상의 값이 넘어가지 않게 해주는 함수가 필요하다.

In [ ]:
# 8.50 타겟 텍스쳐를 업데이트하기 위한 함수 정의
def get_outputs(image):
    image_batch = tf.expand_dims(image, axis=0)
    output = model(preprocess_input(image_batch * 255.0))
    outputs = [gram_matrix(out) for out in output]
    return outputs
  
def get_loss(outputs, style_outputs):
    return tf.reduce_sum([tf.reduce_mean((o-s)**2) for o,s in zip(outputs, style_outputs)])
  
def clip_0_1(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

타깃 텍스터체어 Gram Matrix를 구하는 함수는 get_outputs(image)이다. 그리고 타깃 텍스처와 원본 텍스처의 Gram matrix 값의 MSE를 구하는 함수는 get_loss(outputs, style_outputs)로 각 레이어의 MSE를 구한 다음 합쳐서 최종 손실을 반환한다. clip_0_1(image)는 이미지의 픽셀 컬러값이 0.0에서 1.0 사이가 되도록 벗어나는 값을 잘라주는 역할이다.

지금까지는 tf.keras를 이용해 model을 정의하고 fit()을 통해 가중치가 주어진 과제를 학습시켰다. 그러나 여기에서는 학습해야할 가중치가 존재하지 않는다. 오직 2개의 이미지와 그 Gram matrix의 차이인 MSE 뿐이다.

텐서플로의 **GradientTape**은 이런 상황에 대한 간편한 해결책이다. 어떤 식이 들억다ㅓ라도 자동 미분을 통해 입력에 대한 손실을 구한 다음 다른 변수에 대한 Gradien(기울기)를 계산한다. 여기서 다른 변수는 입력이 될 수도 있고 가중치가 될 수도 있다. 중요한 점은 GradientTape의 계산 과정 안에 묶인 변수에 대한 Gradient여야한다는 점이다.

Gradient를 안다는 것은 입력과 가중치가 손실 함수에 대해 얼마나 좋은지 혹은 나쁜지를 안다는 것과 동일하다. 가중치에 대한 Gradient를 구했다면 가중치에 Gradient를 적용해서 좀 더 좋은 결과를 나오게 할 수 있고, 입력에 대한 Gradient를 구했다면 마찬가지로 입력에 Gradient를 적용할 수 있다.

In [ ]:
# 8.51 tf.function과 GradientTape을 이용한 이미지 업데이트 함수 정의
opt = tf.optimizers.Adam(learning_rate=0.2, beta_1=0.99, epsilon=1e-1)

@tf.function()
def train_step(image):
    with tf.GradientTape() as tape:
        outputs = get_outputs(image)
        loss = get_loss(outputs, style_outputs)

    grad = tape.gradient(loss, image)
    opt.apply_gradients([(grad, image)])
    image.assign(clip_0_1(image))

일단 opt는 최적화 함수를 정의했다. 최적화 함수는 GradientTape을 사용할 때 전체적인 학습 속도를 조정하는 역할을 한다. 

train_step(image)함수 바로 위에 @tf.function은 파이썬 문법 중 장식자라는 것이다. 함수나 클래스 앞에 오면서 이름 앞에 @를 붙이면 장식자가 된다. 장식자는 기존의 코드에 간편하게 기능을 추가한다.

tf.function() 함수는 train_step(image) 함수를 인수로 받아 Autograph라는 강력한 기능을 추가한다. Autograph는 파이썬 문법으로 텐서플로의 핵심인 그래프를 컨트롤할 수 있게 해준다.

    with tf.GradientTape() as tape:
        outputs = get_outputs(image)
        loss = get_loss(outputs, style_outputs)

GradientTape()는 보통 with와 함께 사용한다. 여기서 tape이라는 새로운 GradientTape의 인스턴스를 생성해서 참조한다. 앞에서 설명한대로 get_outputs(image), get_loss(outputs, style_outputs) 함수는 tf.function 장식자를 쓰지 않았지만 호출한 함수에 장식자가 붙었기 때문에 텐서프로의 그래프에 자동으로 포함된다. 

    grad = tape.gradient(loss, image)
    opt.apply_gradients([grad, image])
    image.assign(clip_0_1(image))

tape.gradient(loss, image)는 with 구문안에서 발생한 계산을 추적해서 입력값인 image에 대한 loss의 gradient를 계산한다. 이렇게 계산된 gradient는 변수 grad에 저장되고 Adam optimizer를 통해 image에 영향을 준다.즉 입력값인 image는 이 계산을 변화한다. 그리고 마지막 줄 clip_0_1(image)함수의 계산 결과를 image에 다시 넣어서 컬러값이 0.0, 1.0 사이에 머물게 한다.

In [ ]:
# 8.52 텍스쳐 합성 알고리즘 실행
import IPython.display as display
import time
import imageio

start = time.time()

image = tf.Variable(target_image)

epochs = 50
steps_per_epoch = 100

step = 0
for n in range(epochs):
    for m in range(steps_per_epoch):
        step += 1
        train_step(image)
    if n % 5 == 0 or n == epochs - 1:
        imageio.imwrite('style_epoch_{0}.png'.format(n), image.read_value().numpy())
    display.clear_output(wait=True)
    plt.imshow(image.read_value())
    plt.title("Train step: {}".format(step))
    plt.show()

end = time.time()
print("Total time: {:.1f}".format(end-start))

가장 먼저 새로운 텍스처 출력을 나타내고 이전 텍스처 출력을 지우기 위해 IPython.display 에서 display를 임포트 했다. 그리고 실행시간을 추적하기 위해 time, 합성된 텍스처 이미지를 저장하기 위해 imageio를 임포트 했다.

    image = tf.Variable(target_image)

타깃 텍스처에 tf.Variable을 씌워서 image라는 변수로 저장한다. 텐서플로에서 그래프 연산을 하는 tensor는 tf.Variable  이나 tf.Constant등에 저장돼야한다.

for 문에서는 에포크 당 100step 씩 train_stap(image) 함수를 실행시킨다.

이미지에 생기는 자글자글한 노이즈를 개선하기 위해서 전체 손실에 variation loss를 추가해볼 수 있다. variation loss란 어떤 픽셀과 바로 옆에 인접한 픽셀의 창이이다. 이 차이가 작을수록 이미지는 매끄럽게 보일 것이다.

In [ ]:
# 8.53 varitation loss 함수 정의
def high_pass_x_y(image):
    x_var = image[:,1:,:] - image[:,:-1,:]
    y_var = image[1:,:,:] - image[:-1,:,:]
    return x_var, y_var

def total_variation_loss(image):
    x_deltas, y_deltas = high_pass_x_y(image)
    return tf.reduce_mean(x_deltas**2) + tf.reduce_mean(y_deltas**2)

high_pass_x_y(image) 함수에서는 입력된 image의 x축 방향과 y축 방향의 차이를 구한다. 이렇게 구한 x_var는 image가 (224,224,3)의 shape일 때 (224,224,3)이 되고 y_var는 (224,224,3)으로 각각 x,y방향으로 1픽셀씩 작은 image가 된다. total_variation_loss(image) 함수에서는 이렇게 구한 x,y축 방향의 차이를 제곱해서 평균을 낸 다음 합해서 반환한다.

왜 variation loss가 필요한지 원본 텍스처와 타깃 텍스처, 그리고 랜덤 노이즈 텍스처의 variation loss와 비교해 보자.

In [ ]:
# 8.54 variation loss 비교
print('target   :', total_variation_loss(image.read_value()))
print('noise    :', total_variation_loss(tf.random.uniform(style_image.shape)))
print('original :', total_variation_loss(style_image))

In [ ]:
# 8.55 variation loss를 loss 계산식에 추가, 각 loss의 가중치 추가
total_variation_weight = 1e9
style_weight = 1e-1

@tf.function()
def train_step(image):
    with tf.GradientTape() as tape:
        outputs = get_outputs(image)
        loss = style_weight * get_loss(outputs, style_outputs)
        loss += total_variation_weight * total_variation_loss(image)

    grad = tape.gradient(loss, image)
    opt.apply_gradients([(grad, image)])
    image.assign(clip_0_1(image))

In [ ]:
# 8.56 variation loss를 추가한 텍스쳐 합성 알고리즘 실행
start = time.time()

target_image = tf.random.uniform(style_image.shape)
image = tf.Variable(target_image)

epochs = 50
steps_per_epoch = 100

step = 0
for n in range(epochs):
    for m in range(steps_per_epoch):
        step += 1
        train_step(image)
    if n % 5 == 0 or n == epochs - 1:
        imageio.imwrite('style_variation_epoch_{0}.png'.format(n), image.read_value().numpy())
    display.clear_output(wait=True)
    plt.imshow(image.read_value())
    plt.title("Train step: {}".format(step))
    plt.show()

end = time.time()
print("Total time: {:.1f}".format(end-start))

In [ ]:

# 8.57 원본과 타겟의 variation loss 비교
print('target   :', total_variation_loss(image.read_value()))
print('original :', total_variation_loss(style_image))

## 8.3.2 컨볼루션 신경망을 사용한 신경 스타일 전이

신경 스타일 전이는 위에서 배운 Gram matrix를 이용한 텍스처 합성에 한가지를 더 추가한 것이빈다. 바로 content 텍스처 이다. 타겟 텍스처를 만들기 위해서 style 텍스처와 Grammatrix의 MSE를 구하고, content 텍스처와는 픽셀 값의 차이인 MSE를 구한다. 여기서 사용하는 추출값을 위한 레이어는 서로 다르게 설정할 수 있다.

styleloss를 위해 다섯개의 레이어에서 gram matrix를 계산하는 데 비해 content loss를 위해서는 뒤쪽의 한 개 레이어에서 얻은 특징 추출값만 사용한다. 

content 복원에서 앞쪽 레이어의 특징 추출로 만든 복원 결과는 거의 완벽하지만 픽셀값을 복사하는 수준에 그칠 뿐이고, 뒤쪽 레이어의 특징 추출로 만든 복원 결과는 그림의 세부 픽셀이 깨지지만 전체적인 구조는 유지된다. 신경 스타일 전이로 얻고 싶은 결과는 단순한 픽셀의 복사가 아닌 content 구조의 모사이기 때문에 뒤쪽 레이어를 사용한다.

style loss를 구할 때처럼 content loss를 구할 때 앞쪽의 레이어의 특징 추출을 같이 사용할수도 있다. 다만 이 경우 content 이미지의 세부를 그대로 복사하는 결과를 불러올 수 있기 때문에 원본 논문에서도 content loss를 계산하기 위해서는 뒤쪽의 레이어 중 한 개의 특징 추출만 사용했다. 

In [ ]:

# 8.58 content 텍스쳐 불러오기
import matplotlib.pyplot as plt
import cv2

content_path = tf.keras.utils.get_file('content.jpg', 'http://bit.ly/2mAfUX1')

content_image = plt.imread(content_path)
max_dim = 512
long_dim = max(content_image.shape[:-1])
scale = max_dim / long_dim
new_height = int(content_image.shape[0] * scale)
new_width = int(content_image.shape[1] * scale)

content_image = cv2.resize(content_image, dsize=(new_width, new_height))
content_image = content_image / 255.0
plt.figure(figsize=(8,8))
plt.imshow(content_image)

한 가지 주의해야 할 점은 **content 텍스처와 타깃 텍스처의 크기가 같아야한다!!!** 이 둘은 서로 특징 추출값의 픽셀을 MSE로 비교하기 때문에 크기가 다르면 안된다. 반면 style 텍스처는 타깃 텍스처와 크기가 달라도 상관 없다. Gram matrix 계산값은 각 레이어의 채널수 * 채널수 만큼의 값을 서로 비교하기 때문이다.

In [ ]:
# 8.59 content 특징 추출 모델 만들기
content_batch = content_image.astype('float32')
content_batch = tf.expand_dims(content_batch, axis=0)

content_layers = ['block5_conv2']

vgg.trainable = False
outputs = [vgg.get_layer(name).output for name in content_layers]
model_content = tf.keras.Model([vgg.input], outputs)
content_output = model_content(preprocess_input(content_batch * 255.0))

content 특징 추출을 위해 선택한 레이어는 block5_conv1 바로 뒤에 있는 block5_conv2이다. 위에서 style 특징을 추출하는 모델과 별도의 모델을 만들어서 model_content에 저장하고, 이 모델을 사용해 content 텍스처에서 미리 특징을 추출해서 content_output 변수에 저장해놓는다. style_outputs 처럼 이 값도 여기서 한 번 계산하면 바뀌지 않고 계속 사용되는 값이다.

In [ ]:
# 8.60 content output, loss 함수 정의
def get_content_output(image):
    image_batch = tf.expand_dims(image, axis=0)
    output = model_content(preprocess_input(image_batch * 255.0))
    return output

def get_content_loss(image, content_output):
    return tf.reduce_sum(tf.reduce_mean(image-content_output)**2)

get_content_loss(image, content_output) 함수에서 타깃 텍스처와 content 텍스처의 MSE를 구한다.

그 다음 content loss를 계산식에 추가하고, 모델의 하이퍼파리마티인 Adam optimizer의 학습률과 각 loss의 가중치들을 신경 스타일 전이 과제에 맞게 조정한다. 

In [ ]:
# 8.61 content loss를 loss 계산식에 추가
opt = tf.optimizers.Adam(learning_rate=0.001, beta_1=0.99, epsilon=1e-1)

total_variation_weight = 1e9
style_weight = 1e-2
content_weight = 1e4

@tf.function()
def train_step(image):
    with tf.GradientTape() as tape:
        outputs = get_outputs(image)
        output2 = get_content_output(image)
        loss = style_weight * get_loss(outputs, style_outputs)
        loss += content_weight * get_content_loss(output2, content_output)
        loss += total_variation_weight * total_variation_loss(image)

    grad = tape.gradient(loss, image)
    opt.apply_gradients([(grad, image)])
    image.assign(clip_0_1(image))

In [ ]:
# 8.62 Neural Style Transfer 실행
start = time.time()

# target_image = tf.random.uniform(content_image.shape)
image = tf.Variable(content_image.astype('float32'))

epochs = 20
steps_per_epoch = 100

step = 0
for n in range(epochs):
    for m in range(steps_per_epoch):
        step += 1
        train_step(image)
        print(".", end='')
    if n % 5 == 0 or n == epochs - 1:
        imageio.imwrite('style_{0}_content_{1}_transfer_epoch_{2}.png'.format(style_weight, content_weight, n), image.read_value().numpy())
    display.clear_output(wait=True)
    plt.figure(figsize=(8,8))
    plt.imshow(image.read_value())
    plt.title("Train step: {}".format(step))
    plt.show()

end = time.time()
print("Total time: {:.1f}".format(end-start))

In [ ]:
style_image = plt.imread(style_path)
style_image = cv2.resize(style_image, dsize=(224, 224))
style_image = style_image / 255.0

style_batch = style_image.astype('float32')
style_batch = tf.expand_dims(style_batch, axis=0)
style_output = model(preprocess_input(style_batch * 255.0))

plt.figure(figsize=(16,16))

for c in range(style_output[0].shape[-1]):
    plt.subplot(8,8,c+1)
    plt.axis('off')
    plt.imshow(tf.squeeze(style_output[0][:,:,:,c], 0), cmap='gray')